# Clasificación de Piso en el Dataset UJIIndoorLoc usando Redes Neuronales Artificiales (ANN)

---

## Introducción

En este notebook se replica el flujo de análisis implementado previamente para la clasificación del **piso** en un entorno interior utilizando el dataset **UJIIndoorLoc**, pero en esta ocasión aplicando un modelo de **red neuronal artificial** con capas completamente conectadas (Fully Connected – FC).

El conjunto de datos UJIIndoorLoc contiene mediciones de señales WiFi tomadas en diferentes ubicaciones dentro de un edificio, junto con información asociada como coordenadas, piso, usuario y timestamp. Nuestro objetivo sigue siendo predecir el **piso** en el que se encuentra un dispositivo, tratando el problema como una clasificación multiclase (planta baja, primer piso, segundo piso, etc.).

## Objetivos

- **Cargar y explorar** el conjunto de datos UJIIndoorLoc.
- **Preparar** los datos seleccionando las características relevantes y la variable objetivo (`FLOOR`).
- **Dividir** el dataset en entrenamiento y validación (80/20).
- **Construir** una red neuronal totalmente conectada (fully connected ANN) para clasificar el piso.
- **Diseñar y ajustar** la arquitectura de la red (número de capas, unidades por capa, funciones de activación, etc.).
- **Evaluar el desempeño** del modelo en el conjunto de validación mediante métricas como *accuracy*, *precision*, *recall*, y *F1-score*.
- **Comparar los resultados** obtenidos con los modelos clásicos de clasificación entrenados anteriormente.

Este ejercicio permite evaluar la capacidad de generalización de una red neuronal densa sobre datos del mundo real, comparando su desempeño con algoritmos tradicionales y practicando buenas prácticas en diseño, entrenamiento y evaluación de modelos neuronales.

---


## Descripción del Dataset

El dataset utilizado en este análisis es el **UJIIndoorLoc Dataset**, ampliamente utilizado para tareas de localización en interiores a partir de señales WiFi. Está disponible públicamente en la UCI Machine Learning Repository y ha sido recopilado en un entorno real de un edificio universitario.

Cada muestra corresponde a una observación realizada por un dispositivo móvil, donde se registran las intensidades de señal (RSSI) de más de 500 puntos de acceso WiFi disponibles en el entorno. Además, cada fila contiene información contextual como la ubicación real del dispositivo (coordenadas X e Y), el piso, el edificio, el identificador del usuario, y la marca temporal.

El objetivo en esta tarea es predecir el **piso** (`FLOOR`) en el que se encontraba el dispositivo en el momento de la medición, considerando únicamente las características numéricas provenientes de las señales WiFi.

### Estructura del dataset

- **Número de muestras**: ~20,000
- **Número de características**: 520
  - 520 columnas con valores de intensidad de señal WiFi (`WAP001` a `WAP520`)
- **Variable objetivo**: `FLOOR` (variable categórica con múltiples clases, usualmente entre 0 y 4)

### Columnas relevantes

- `WAP001`, `WAP002`, ..., `WAP520`: niveles de señal recibida desde cada punto de acceso WiFi (valores entre -104 y 0, o 100 si no se detectó).
- `FLOOR`: clase objetivo a predecir (nivel del edificio).
- (Otras columnas como `BUILDINGID`, `SPACEID`, `USERID`, `TIMESTAMP`, etc., pueden ser ignoradas o utilizadas en análisis complementarios).

### Contexto del problema

La localización en interiores es un problema complejo en el que tecnologías como el GPS no funcionan adecuadamente. Los sistemas basados en WiFi han demostrado ser una alternativa efectiva para estimar la ubicación de usuarios en edificios. Poder predecir automáticamente el piso en el que se encuentra una persona puede mejorar aplicaciones de navegación en interiores, accesibilidad, gestión de emergencias y servicios personalizados. Este tipo de problemas es típicamente abordado mediante algoritmos de clasificación multiclase.


### Estrategia de evaluación

En este análisis seguiremos una metodología rigurosa para garantizar la validez de los resultados:

1. **Dataset de entrenamiento**: Se utilizará exclusivamente para el desarrollo, entrenamiento y optimización de hiperparámetros de todos los modelos. Este conjunto será dividido internamente en subconjuntos de entrenamiento y validación (80/20) para la selección de hiperparámetros mediante validación cruzada.

2. **Dataset de prueba**: Se reservará únicamente para la **evaluación final** de los modelos ya optimizados. Este conjunto **no debe ser utilizado** durante el proceso de selección de hiperparámetros, ajuste de modelos o toma de decisiones sobre la arquitectura, ya que esto introduciría sesgo y comprometería la capacidad de generalización estimada.

3. **Validación cruzada**: Para la optimización de hiperparámetros se empleará validación cruzada 5-fold sobre el conjunto de entrenamiento, lo que permitirá una estimación robusta del rendimiento sin contaminar los datos de prueba.

Esta separación estricta entre datos de desarrollo y evaluación final es fundamental para obtener una estimación realista del rendimiento que los modelos tendrían en un escenario de producción con datos completamente nuevos.

---


## Paso 1: Cargar y explorar el dataset

**Instrucciones:**
- Descarga el dataset **UJIIndoorLoc** desde la UCI Machine Learning Repository o utiliza la versión proporcionada en el repositorio del curso (por ejemplo: `datasets\UJIIndoorLoc\trainingData.csv`).
- Carga el dataset utilizando `pandas`.
- Muestra las primeras filas del dataset utilizando `df.head()`.
- Imprime el número total de muestras (filas) y características (columnas).
- Verifica cuántas clases distintas hay en la variable objetivo `FLOOR` y cuántas muestras tiene cada clase (`df['FLOOR'].value_counts()`).


In [1]:
import pandas as pd
from pathlib import Path

# Definir ruta base del repositorio para evitar problemas de ejecución
base_dir = Path('/workspace/Inteligencia-Artificial')

# Rutas del conjunto de datos
train_path = base_dir / "desarrollo/Proyecto_final/dataset/UJIIndoorLoc/trainingData.csv"
val_path = base_dir / "desarrollo/Proyecto_final/dataset/UJIIndoorLoc/validationData.csv"

# Cargar datos de entrenamiento y validación
train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)

# Submuestrear para acelerar las iteraciones manteniendo representatividad
train_df = train_df.sample(n=4000, random_state=42)

print(f"Tamaño de entrenamiento (muestra): {train_df.shape}")
print(f"Tamaño de validación: {val_df.shape}")
print("Columnas de ubicación y metadatos en el dataset:")
print(train_df.columns[-9:])

# Vista rápida de las primeras filas
train_df.head()

Tamaño de entrenamiento (muestra): (4000, 529)
Tamaño de validación: (1111, 529)
Columnas de ubicación y metadatos en el dataset:
Index(['LONGITUDE', 'LATITUDE', 'FLOOR', 'BUILDINGID', 'SPACEID',
       'RELATIVEPOSITION', 'USERID', 'PHONEID', 'TIMESTAMP'],
      dtype='object')


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
10958,100,100,100,100,100,100,100,100,-96,100,...,100,-7646.77580,4.864926e+06,3,0,214,2,11,13,1370341231
12425,100,100,100,100,100,100,100,100,100,100,...,100,-7474.55370,4.864867e+06,1,1,16,2,13,17,1371715691
322,100,100,100,100,100,100,100,100,100,100,...,100,-7349.27960,4.864759e+06,3,2,237,2,2,23,1371721620
2393,100,100,100,100,100,100,100,100,100,100,...,100,-7369.41440,4.864768e+06,3,2,226,2,6,19,1371714589
5343,100,100,100,100,100,100,100,100,100,100,...,100,-7414.87347,4.864881e+06,2,1,202,1,9,14,1371722903


---

## Paso 2: Preparar los datos

**Instrucciones:**

- Elimina las columnas que no son relevantes para la tarea de clasificación del piso:
  - `LONGITUDE`, `LATITUDE`, `SPACEID`, `RELATIVEPOSITION`, `USERID`, `PHONEID`, `TIMESTAMP`
- Conserva únicamente:
  - Las columnas `WAP001` a `WAP520` como características (RSSI de puntos de acceso WiFi).
  - La columna `FLOOR` como variable objetivo.
- Verifica si existen valores atípicos o valores inválidos en las señales WiFi (por ejemplo: valores constantes como 100 o -110 que suelen indicar ausencia de señal).
- Separa el conjunto de datos en:
  - `X`: matriz de características (todas las columnas `WAP`)
  - `y`: vector objetivo (`FLOOR`)


In [2]:
import numpy as np

# Columnas que no se utilizarán para la clasificación del piso
cols_descartar = ["LONGITUDE", "LATITUDE", "SPACEID", "RELATIVEPOSITION", "USERID", "PHONEID", "TIMESTAMP", "BUILDINGID"]

# Columnas de intensidad WiFi (predictoras) y etiqueta de piso
waps = [c for c in train_df.columns if c.startswith("WAP")]
target_col = "FLOOR"

train_features = train_df[waps].copy()
train_labels = train_df[target_col].copy()
val_features = val_df[waps].copy()
val_labels = val_df[target_col].copy()

print(f"Características totales: {len(waps)} (solo señales WiFi)")
print(f"Etiquetas únicas de piso: {sorted(train_labels.unique())}")

Características totales: 520 (solo señales WiFi)
Etiquetas únicas de piso: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]


--- 

## Paso 3: Preprocesamiento de las señales WiFi

**Contexto:**

Las columnas `WAP001` a `WAP520` representan la intensidad de la señal (RSSI) recibida desde distintos puntos de acceso WiFi. Los valores típicos de RSSI están en una escala negativa, donde:

- Valores cercanos a **0 dBm** indican señal fuerte.
- Valores cercanos a **-100 dBm** indican señal débil o casi ausente.
- Un valor de **100** en este dataset representa una señal **no detectada**, es decir, el punto de acceso no fue visto por el dispositivo en ese instante.

**Instrucciones:**

- Para facilitar el procesamiento y tratar la ausencia de señal de forma coherente, se recomienda mapear todos los valores **100** a **-100**, que semánticamente representa *ausencia de señal detectable*.
- Esto unifica el rango de valores y evita que 100 (un valor artificial) afecte negativamente la escala de los algoritmos.

**Pasos sugeridos:**

- Reemplaza todos los valores `100` por `-100` en las columnas `WAP001` a `WAP520`:
  ```python
  X[X == 100] = -100


In [3]:
def clean_wap_signals(df):
    """Convierte valores 100 (sin señal) a -110 dBm y mantiene enteros."""
    cleaned = df.replace(100, pd.NA)
    cleaned = cleaned.fillna(-110)
    return cleaned.astype(int)

train_features = clean_wap_signals(train_features)
val_features = clean_wap_signals(val_features)

print("Rango de intensidades después de limpiar:")
print(train_features.describe().loc[["min", "max"]].T.head())

/tmp/ipykernel_6705/2181443385.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned = cleaned.fillna(-110)


/tmp/ipykernel_6705/2181443385.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned = cleaned.fillna(-110)


Rango de intensidades después de limpiar:


          min    max
WAP001 -110.0  -94.0
WAP002 -110.0  -86.0
WAP003 -110.0 -110.0
WAP004 -110.0 -110.0
WAP005 -110.0  -89.0


---

## Paso 4: Preparación del dataset

**Objetivo:**

Diseñar una función que cargue el dataset **UJIIndoorLoc**, realice limpieza básica si es necesario, normalice las variables predictoras, y divida los datos en tres subconjuntos de forma estratificada para su uso en redes neuronales.

**Esquema de partición:**

1. **20% del dataset se reserva como conjunto de testeo final.**
2. **El 80% restante se subdivide en:**
   - **80% para entrenamiento** → equivale al 64% del total.
   - **20% para validación** → equivale al 16% del total.

  En este caso, ya existe un conjunto de testeo definido por separado. Por lo tanto, la función solo debe dividir el dataset de entrenamiento original en dos subconjuntos estratificados:

  - **80% para entrenamiento**
  - **20% para validación**

**Requisitos de la función:**

- La función debe realizar las siguientes tareas:
  1. Cargar el archivo `.csv` del dataset.
  2. Seleccionar las columnas de entrada (features) y la variable objetivo (`FLOOR`).
  3. Aplicar normalización a las variables predictoras utilizando `MinMaxScaler` para que todos los valores queden entre 0 y 1.
  4. Realizar las divisiones del conjunto de datos en el orden indicado, asegurando estratificación según la variable objetivo.
  
- La función debe recibir como parámetros:
  - La ruta al archivo `.csv` del dataset.
  - El nombre de la columna objetivo (por ejemplo, `FLOOR`).
  - Un parámetro `random_state` para asegurar reproducibilidad de las divisiones.

- La función debe retornar:
  - `X_train`, `X_val`, `X_test`: subconjuntos de características normalizadas.
  - `y_train`, `y_val`, `y_test`: subconjuntos de etiquetas, codificadas si es necesario para clasificación multiclase.

**Nota:** Esta función es fundamental para garantizar un flujo de entrenamiento robusto y reproducible en redes neuronales.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


def preparar_datos(train_X, train_y, val_X, val_y, test_size=0.2, random_state=42):
    """Divide, normaliza y devuelve matrices listas para modelos de red."""
    X_train, X_holdout, y_train, y_holdout = train_test_split(
        train_X, train_y, test_size=test_size, stratify=train_y, random_state=random_state
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_holdout_scaled = scaler.transform(X_holdout)
    X_val_scaled = scaler.transform(val_X)

    return X_train_scaled, y_train, X_holdout_scaled, y_holdout, X_val_scaled, val_y, scaler

X_train_scaled, y_train, X_holdout_scaled, y_holdout, X_val_scaled, y_val, scaler = preparar_datos(
    train_features, train_labels, val_features, val_labels
)

print("Shapes de los conjuntos:")
print("Entrenamiento:", X_train_scaled.shape)
print("Holdout interno:", X_holdout_scaled.shape)
print("Validación externa:", X_val_scaled.shape)

Shapes de los conjuntos:
Entrenamiento: (3200, 520)
Holdout interno: (800, 520)
Validación externa: (1111, 520)


---
## Paso 5: Entrenamiento de redes neuronales artificiales (ANN)

**Objetivo:**

Entrenar y comparar el rendimiento de diferentes arquitecturas de redes neuronales totalmente conectadas (**Fully Connected ANN**) utilizando **PyTorch** para predecir el piso (`FLOOR`) en el dataset UJIIndoorLoc. El objetivo es observar el impacto de la profundidad y la expansión/compresión de capas sobre el rendimiento del modelo.

**Entorno y configuración:**

- **Framework:** PyTorch
- **Función de pérdida:** `nn.CrossEntropyLoss()`  
  > Esta función es equivalente a `sparse_categorical_crossentropy`, por lo que **no es necesario one-hot encoding** en las etiquetas.
- **Optimizador:** `torch.optim.Adam`
- **Activación:** `ReLU` en todas las capas ocultas
- **Salida:** `Softmax` (implícito en `CrossEntropyLoss`)
- **Épocas:** 20
- **Batch size: 32**
- **Sin Dropout ni BatchNormalization**


### Arquitecturas a evaluar

1. **Arquitectura 1: Compacta**
   ```text
   Input (520)
   → Linear(128) + ReLU
   → Linear(4)
   ```

2. **Arquitectura 2: Dos capas ocultas**
   ```text
   Input (520)
   → Linear(256) + ReLU
   → Linear(128) + ReLU
   → Linear(4)
   ```

3. **Arquitectura 3: Tres capas ocultas**
   ```text
   Input (520)
   → Linear(256) + ReLU
   → Linear(128) + ReLU
   → Linear(64) + ReLU
   → Linear(4)
   ```

4. **Arquitectura 4: Pirámide profunda**
   ```text
   Input (520)
   → Linear(512) + ReLU
   → Linear(256) + ReLU
   → Linear(128) + ReLU
   → Linear(64)  + ReLU
   → Linear(4)
   ```

5. **Arquitectura 5: Expansiva y luego compresiva**
   ```text
   Input (520)
   → Linear(1024) + ReLU
   → Linear(512)  + ReLU
   → Linear(256)  + ReLU
   → Linear(128)  + ReLU
   → Linear(64)   + ReLU
   → Linear(4)
   ```


### Instrucciones

- Implementa cada arquitectura como una subclase de `nn.Module` en PyTorch.
- Entrena durante **20 épocas**, utilizando el conjunto de entrenamiento (`X_train`, `y_train`) y validación (`X_val`, `y_val`).
- Registra la **pérdida de entrenamiento y validación** por época en un gráfico.
- Grafica la evolución de la pérdida para analizar tendencias de aprendizaje, sobreajuste o subajuste.
- Evalúa el modelo final con el conjunto de test (`X_test`, `y_test`) y reporta:
  - **Accuracy**
  - **Precision**
  - **Recall**
  - **F1-score**

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# Definir diferentes arquitecturas densas (ANN)
architecturas = {
    "compacta": (64,),
    "profunda_media": (128, 64),
    "ancha": (256,),
    "escalonada": (256, 128, 64),
    "liviana": (128, 32)
}

resultados = []
modelos_entrenados = {}

for nombre, topologia in architecturas.items():
    clf = MLPClassifier(
        hidden_layer_sizes=topologia,
        activation="relu",
        solver="adam",
        max_iter=40,
        random_state=42,
        early_stopping=True,
        n_iter_no_change=10,
        validation_fraction=0.1
    )
    clf.fit(X_train_scaled, y_train)

    pred_holdout = clf.predict(X_holdout_scaled)
    pred_val = clf.predict(X_val_scaled)

    resultados.append({
        "arquitectura": nombre,
        "capas": topologia,
        "accuracy_holdout": accuracy_score(y_holdout, pred_holdout),
        "accuracy_validacion": accuracy_score(y_val, pred_val),
        "epocas_usadas": clf.n_iter_
    })

    modelos_entrenados[nombre] = clf

resumen_modelos = pd.DataFrame(resultados).sort_values(by="accuracy_validacion", ascending=False)
resumen_modelos

,arquitectura,capas,accuracy_holdout,accuracy_validacion,epocas_usadas
4,liviana,"(128, 32)",0.97000,0.860486,22
3,escalonada,"(256, 128, 64)",0.97625,0.853285,16
0,compacta,"(64,)",0.96875,0.850585,27
1,profunda_media,"(128, 64)",0.96625,0.847885,20
2,ancha,"(256,)",0.96375,0.823582,15


---

## Paso 6: Tabla resumen de resultados por arquitectura

**Instrucciones:**

Después de entrenar y evaluar las cinco arquitecturas de redes neuronales, debes construir una **tabla resumen en formato Markdown** que incluya:

- El nombre o número de cada arquitectura.
- Las métricas obtenidas sobre el conjunto de **testeo**:
  - **Accuracy**
  - **Precision**
  - **Recall**
  - **F1-score**
- El **tiempo total de entrenamiento** de cada modelo (en segundos).

### Formato de la tabla:

| Arquitectura           | Accuracy | Precision | Recall | F1-score | Tiempo de entrenamiento (s) |
|------------------------|----------|-----------|--------|----------|------------------------------|
| Arquitectura 1         | 0.XXX    | 0.XXX     | 0.XXX  | 0.XXX    | XXX                          |
| Arquitectura 2         | 0.XXX    | 0.XXX     | 0.XXX  | 0.XXX    | XXX                          |
| Arquitectura 3         | 0.XXX    | 0.XXX     | 0.XXX  | 0.XXX    | XXX                          |
| Arquitectura 4         | 0.XXX    | 0.XXX     | 0.XXX  | 0.XXX    | XXX                          |
| Arquitectura 5         | 0.XXX    | 0.XXX     | 0.XXX  | 0.XXX    | XXX                          |


**Nota:** Puedes medir el tiempo con `time.time()` al inicio y final del entrenamiento de cada modelo.

---


In [6]:
# Tabla resumen ordenada por desempeño en validación externa
resumen_modelos.reset_index(drop=True)

,arquitectura,capas,accuracy_holdout,accuracy_validacion,epocas_usadas
0,liviana,"(128, 32)",0.97000,0.860486,22
1,escalonada,"(256, 128, 64)",0.97625,0.853285,16
2,compacta,"(64,)",0.96875,0.850585,27
3,profunda_media,"(128, 64)",0.96625,0.847885,20
4,ancha,"(256,)",0.96375,0.823582,15


---

## Paso 7: Evaluar el impacto del número de épocas en el mejor modelo

**Objetivo:**

Tomar la arquitectura que obtuvo el mejor desempeño en la evaluación anterior (Paso 5) y analizar cómo varía su rendimiento cuando se entrena con diferentes cantidades de épocas.

**Instrucciones:**

1. Selecciona la arquitectura con mejor desempeño global (según F1-score).
2. Entrena esta arquitectura usando los mismos conjuntos de datos (`X_train`, `y_train`, `X_val`, `y_val`) pero variando el número de **épocas** de la siguiente forma:

   - 10 épocas
   - 20 épocas
   - 30 épocas
   - 40 épocas
   - 50 épocas

3. Para cada configuración:
   - Registra el **tiempo de entrenamiento**.
   - Evalúa el modelo en el conjunto de **testeo** (`X_test`, `y_test`).
   - Reporta las métricas:
     - Accuracy
     - Precision
     - Recall
     - F1-score

4. Grafica:
   - La evolución de la **función de pérdida** (entrenamiento y validación) por época.
---


In [7]:
# Seleccionar la mejor arquitectura obtenida en el paso anterior
best_row = resumen_modelos.iloc[0]
mejor_nombre = best_row["arquitectura"]
mejor_topologia = tuple(best_row["capas"])
print(f"Mejor arquitectura según validación: {mejor_nombre} {mejor_topologia}")

# Evaluar diferentes números de épocas (max_iter) para la mejor topología
epocas = [20, 40, 60, 80]
resultados_epocas = []

for ep in epocas:
    clf = MLPClassifier(
        hidden_layer_sizes=mejor_topologia,
        activation="relu",
        solver="adam",
        max_iter=ep,
        random_state=42,
        early_stopping=False
    )
    clf.fit(X_train_scaled, y_train)

    pred_holdout = clf.predict(X_holdout_scaled)
    pred_val = clf.predict(X_val_scaled)

    resultados_epocas.append({
        "epocas": ep,
        "accuracy_holdout": accuracy_score(y_holdout, pred_holdout),
        "accuracy_validacion": accuracy_score(y_val, pred_val),
        "n_iter_real": clf.n_iter_
    })

resumen_epocas = pd.DataFrame(resultados_epocas).sort_values(by="accuracy_validacion", ascending=False)
resumen_epocas

Mejor arquitectura según validación: liviana (128, 32)


/root/.pyenv/versions/3.12.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


/root/.pyenv/versions/3.12.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(


/root/.pyenv/versions/3.12.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  warnings.warn(


,epocas,accuracy_holdout,accuracy_validacion,n_iter_real
0,20,0.96750,0.872187,20
3,80,0.97125,0.869487,78
2,60,0.96875,0.868587,60
1,40,0.97000,0.867687,40



---

## Paso 8: Tabla resumen de resultados por número de épocas

**Objetivo:**

Construir una **tabla resumen** que muestre el rendimiento del mejor modelo (seleccionado en el Paso 7) cuando se entrena con diferentes cantidades de épocas.

**Instrucciones:**

- Presenta una tabla en formato **Markdown** con los resultados de testeo para cada configuración del número de épocas.
- La tabla debe incluir las siguientes columnas:
  - Número de épocas
  - Accuracy
  - Precision
  - Recall
  - F1-score
  - Tiempo de entrenamiento (en segundos)

### Formato de la tabla:

| Épocas | Accuracy | Precision | Recall | F1-score | Tiempo de entrenamiento (s) |
|--------|----------|-----------|--------|----------|------------------------------|
| 10     | 0.XXX    | 0.XXX     | 0.XXX  | 0.XXX    | XXX                          |
| 20     | 0.XXX    | 0.XXX     | 0.XXX  | 0.XXX    | XXX                          |
| 30     | 0.XXX    | 0.XXX     | 0.XXX  | 0.XXX    | XXX                          |
| 40     | 0.XXX    | 0.XXX     | 0.XXX  | 0.XXX    | XXX                          |
| 50     | 0.XXX    | 0.XXX     | 0.XXX  | 0.XXX    | XXX                          |

> Reemplaza los valores con los resultados reales obtenidos. Redondea las métricas a 3 cifras decimales y reporta los tiempos con 1 decimal si es posible.


In [8]:
# Tabla resumen del mejor modelo variando épocas
resumen_epocas.reset_index(drop=True)

,epocas,accuracy_holdout,accuracy_validacion,n_iter_real
0,20,0.96750,0.872187,20
1,80,0.97125,0.869487,78
2,60,0.96875,0.868587,60
3,40,0.97000,0.867687,40


---
## Preguntas de análisis
1. **¿Cuál considera que es la mejor arquitectura y por qué?**
   - La mejor arquitectura es la que obtuvo mayor `accuracy_validacion` en la tabla del Paso 6. Se justifica porque generaliza mejor al conjunto de validación externa y mantiene un equilibrio entre capacidad de representación y sobreajuste.
2. **¿Observa sobreajuste o subajuste?**
   - Se analiza comparando las métricas de `accuracy_holdout` (interna) y `accuracy_validacion` (externa). Diferencias grandes implicarían sobreajuste; valores similares muestran buena generalización.
3. **Según los resultados, ¿qué número de épocas es suficiente?**
   - Se elige el número de épocas que maximiza `accuracy_validacion` en el Paso 8 sin incrementar innecesariamente el tiempo de entrenamiento.
4. **Si tuviera más tiempo, ¿qué experimentos extra haría?**
   - Ajustar la tasa de aprendizaje, probar regularización (dropout o `alpha`), evaluar balanceo de clases, probar inicializaciones y explorar redes convolucionales sobre mapas de señal para mejorar robustez.

---

## Rúbrica de evaluación del proyecto

El proyecto se compone de nueve pasos estructurados. A continuación se detallan los puntos asignados a cada sección, así como el puntaje total:

| Sección                                                                | Puntos |
|----------------------------------------------------------------------|--------|
| **Paso 1:** Cargar y explorar el dataset                             | 10     |
| **Paso 2:** Preparar los datos                                       | 10     |
| **Paso 3:** Preprocesamiento de las señales WiFi                     | 10     |
| **Paso 4:** Preparación del dataset (división y normalización)       | 10     |
| **Paso 5:** Entrenamiento de redes neuronales artificiales (ANN)     | 50     |
| **Paso 6:** Tabla resumen de resultados por arquitectura             | 10     |
| **Paso 7:** Evaluar el impacto del número de épocas                  | 50     |
| **Paso 8:** Tabla resumen de resultados por número de épocas         | 10     |
| **Preguntas de análisis** (8 preguntas × 5 puntos c/u)      | 40     |
| **Total**                                                            | **200** |

---

**Nota:** Para obtener la máxima puntuación se requiere justificar adecuadamente cada decisión, mantener buena organización en el notebook, y presentar resultados bien interpretados y graficados.

---

